In [0]:

import os
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
from pyspark.sql.functions import datediff, col
from pyspark.sql.functions import col, explode_outer, lpad, regexp_replace
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import posexplode, concat, lit
import seaborn as sns


# Display all columns
pd.set_option('display.max_columns', None)

os.environ['KAGGLEHUB_CACHE'] = '/Volumes/workspace/gjain/project_health'
path = kagglehub.dataset_download("bonifacechosen/nhis-healthcare-claims-and-fraud-dataset")
print(path)

In [0]:
# read into a dataframe
raw = spark.read.csv("/Volumes/workspace/gjain/project_health/datasets/bonifacechosen/nhis-healthcare-claims-and-fraud-dataset/versions/3/simulated_healthcare_claims (1).csv", header=True, inferSchema=True).withColumnRenamed("Amount Billed", "paid_amt").withColumnRenamed("Fraud Type", "fraud").withColumnRenamed("Date Admitted", "admission_dt").withColumnRenamed("Date Discharged", "discharge_dt").withColumnRenamed("Patient ID", "patient_id").withColumnRenamed("Age", "age").withColumnRenamed("Gender", "gender")
display(raw)

In [0]:
# Example: Create age feature
df = raw.toPandas()
df['admission_dt'] = pd.to_datetime(df['admission_dt'])
df['discharge_dt'] = pd.to_datetime(df['discharge_dt']) 

print("Number of null ptnt_age values:", df['age'].isnull().sum())

# Feature engineering: use primary diagnosis/procedure codes, gender, LOS, etc.
df['length_of_stay'] = (df['discharge_dt'] - df['admission_dt']).dt.days.astype('int') 
print("Number of null length_of_stay values:", df['length_of_stay'].isnull().sum())
print(f"Maximum value in column {'length_of_stay'}: {df['length_of_stay'].max()}")
print(f"Minimum value in column {'length_of_stay'}: {df['length_of_stay'].min()}")

# # Label encoding for categorical features
#cat_cols = ['gender']
#for col in cat_cols:
#    df[col] = LabelEncoder().fit_transform(df[col])

In [0]:
def find_outliers(df, column, by='age_group'):
    print(f"Outliers in {column} by {by}:")
    for gender in df[by].unique():
        vals = df[df[by] == gender][column]
        q1 = vals.quantile(0.25)
        q3 = vals.quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q1 + 1.5 * iqr
        outliers = df[(df[by] == gender) & ((df[column] < lower) | (df[column] > upper))]
        print(f"counts of outliers in {gender} in {column}: {len(outliers)}")

# Check for outliers in length_of_stay
find_outliers(df, 'length_of_stay', by='age_group')

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

# Create age buckets
age_bins = [0, 18, 30, 45, 60, 75, 100]
age_labels = ['0-17', '18-29', '30-44', '45-59', '60-74', '75+']
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

# Group by age_group and get statistics
summary = df.groupby('age_group')[['length_of_stay']].agg(['mean', 'median', 'count'])
print("Summary statistics by age_group:")
print(summary)

# Boxplot of LOS by age group
plt.figure(figsize=(8,5))
sns.boxplot(x='age_group', y='length_of_stay', data=df)
plt.title("Length of Stay by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Length of Stay (days)")
plt.grid(True)
plt.show()

In [0]:
# First split off the test set
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

# Then split train_val into train and validation
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=42)
# (0.1765 × 0.85 ≈ 0.15, so val is 15% of total data)

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")